In [1]:
from torch.utils.data import DataLoader
from transforms import Rescale, ToTensor
import torchvision.transforms
from dataset import PathfinderDataset
composed = torchvision.transforms.Compose([Rescale(150), ToTensor()])
dataset = PathfinderDataset("baseline", transform=composed)
dataloader = DataLoader(dataset, batch_size=4,shuffle=True, num_workers=4)

In [2]:
import torch
from torch import nn
from hgru import hConvGRUCell, hConvGRU

model = hConvGRU(input_shape=(150,150,1), timesteps=8, filt_size=15)
model = nn.DataParallel(model).cuda()

Training with filter size: 15 x 15


In [3]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss().cuda()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [ ]:
for epoch in range(2):  # loop over the dataset multiple times

    for i, data in enumerate(dataloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs = data['image'].cuda()
        labels = data['label'].cuda()

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model.forward(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        print('[%d, %d/%d] loss: %.6f' % (epoch + 1, i + 1, len(dataloader), loss.item()), end='\r')

print('Finished Training')

In [ ]:
torch.save(model.state_dict(), "model_e2.pkl")